Importamos las librerías que vamos a usar

In [34]:
import pandas as pd
import glob
import re

from datetime import datetime

import numpy as np

### Limpieza tablas padrón Madrid : formato 'long'

Nos hemos propuesto obtener todos los .csv de la carpeta, escritos con una estructura bastante exhaustiva. Nuestra tarea será agruparlos por distrito y sumar la cantidad total de nacionales y extranjeros empadronados por mes (cada .csv contiene un mes)

Vamos a almacenar todos esos dataframes y a iterar sobre ellos, buscando la fecha en el nombre de archivo, ya que todos tienen una estructura tipo 'Rango_Edades_Seccion_*YYYYMM*.csv, para crear una variable con la fecha.

Luego, vamos a sumar las columnas de españoles y extranjeros para ver cuanta gente vive en cada barrio en cada momento.
Hemos tenido que usar un 'contains' y que, a veces, estaba escrito como españoles y otras como espanoles.

Finalmente nos quedamos con el distrito, los nacionales y los extranjeros y añadimos una columna total, agrupando por distrito y añadiendo el distrito 'Madrid' que contenga la suma de todos. Concatenamos esa fila a cada uno de los archivos por mes que tenemos, y le incorporamos el mes en el que ha ocurrido, formateándolo con datetime para que tenga una estructura de DD/MM/YY.

Incorporamos cada uno de estos dataframes a nuestra lista de dataframes y los concatenamos para acabar exportando un archivo .csv del histórico del padrón agrupado por distrito.

In [92]:


# Obtenemos la lista de .csv y creamos una lista para ir almacenando los dataframes
archivos_csv = glob.glob('Rango_Edades_Seccion_*.csv')
dataframes = []

# Ahora vamos a iterar sobre los archivos
for archivo in archivos_csv:
    # Obtenemos la fecha del nombre del arhivo, y leemos el .csv
    fecha = re.search(r'Rango_Edades_Seccion_(\d+)', archivo).group(1)
    data = pd.read_csv(archivo, encoding='latin-1', delimiter=';')
    
    # Vamos a obtener los nombres de las columnas (cada una ha sido escrita de diferente manera, respetando o no la 'ñ', y también vamos a no distinguir entre mayusculas y minusculas)
    columnas_espanoles_hombres = data.columns[data.columns.str.contains('espanoles|españoles', case=False)].tolist()
    columnas_espanoles_mujeres = data.columns[data.columns.str.contains('espanolas|españolas', case=False)].tolist()
    columnas_extranjeros_hombres = data.columns[data.columns.str.contains('extranjeros', case=False)].tolist()
    columnas_extranjeros_mujeres = data.columns[data.columns.str.contains('extranjeras', case=False)].tolist()
    
    # Sumamos los valores para obtener las columnas de 'Nacionales' y 'Extranjeros', y luego nos quedamos solo con las relevantes.
    # Manipulamos los nombres de las columnas para que sigan el estándar que hemos definido con nuestros compañeros

    data['Nacionales'] = data[columnas_espanoles_hombres].sum(axis=1) + data[columnas_espanoles_mujeres].sum(axis=1)
    data['Extranjeros'] = data[columnas_extranjeros_hombres].sum(axis=1) + data[columnas_extranjeros_mujeres].sum(axis=1)
    ag_data = data[['Nacionales', 'Extranjeros', 'DESC_DISTRITO']]
    final = ag_data.copy()
    final['DESC_DISTRITO'] = final['DESC_DISTRITO'].replace({
        '.*FUENCARRAL-EL PARDO.*': 'FUENCARRAL',
        '.*MONCLOA.*': 'MONCLOA',
        '.*PUENTE DE VALLECAS.*': 'PUENTE_VALLECAS',
        '.*SAN BLAS.*': 'SAN:BLAS',
        '.*VILLA DE VALLECAS.*': 'VILLA_VALLECAS',
        '.*CIUDAD LINEAL.*': 'CIUDAD_LINEAL'
    }, regex=True)
    
    # Agrupamos, sumamos por distrito, calculasmos los totales, creamos la fila 'madrid' y la agregamos al dataframe
    agrupado = final.groupby('DESC_DISTRITO').sum()
    
    total_nacionales = agrupado['Nacionales'].sum()
    total_extranjeros = agrupado['Extranjeros'].sum()
    
    madrid_row = pd.DataFrame([[total_nacionales, total_extranjeros]], columns=['Nacionales', 'Extranjeros'], index=['MADRID'])
    agrupado = pd.concat([agrupado, madrid_row])
    
    # Convertimos las columnas a 'int', y formateamos fecha para que acabe con el formato que buscávamos, añadiéndolo al dataframe
    agrupado['nacionales'] = agrupado['Nacionales'].astype(int)
    agrupado['extranjeros'] = agrupado['Extranjeros'].astype(int)
    agrupado['total'] = agrupado['Nacionales'] + agrupado['Extranjeros']
    
    fecha_dt = datetime.strptime(fecha, "%Y%m")
    fecha_formateada = fecha_dt.strftime("%d/%m/%y")
    agrupado['mes'] = fecha_formateada

    #Renombramos el eje a 'distrito' (ya que al estar agrupado por distrito este se situó en el eje)
    agrupado = agrupado.rename_axis('distrito')
    agrupado.index = agrupado.index.str.lower()
    agrupado = agrupado.reset_index()

    # Agregamos
    dataframes.append(agrupado)

# Concatenamos y exportamos :)
resultado = pd.concat(dataframes)

resultado.to_csv('PADRON1222_LONG.csv', index=True)

Para manipular nuestro archivo, lo primero será unificar los distritos para que todos estén en minúscula y sin espacios que puedan confundir las categorías.

In [93]:
data= pd.read_csv('PADRON1222_LONG.csv')
data['distrito'] = data['distrito'].str.strip().str.lower()
data['distrito'].value_counts()


distrito
arganzuela              108
barajas                 108
madrid                  108
villa_vallecas          108
villaverde              108
vicalvaro               108
usera                   108
tetuan                  108
san:blas                108
salamanca               108
retiro                  108
puente_vallecas         108
moratalaz               108
moncloa                 108
latina                  108
hortaleza               108
fuencarral              108
chamberi                108
chamartin               108
centro                  108
carabanchel             108
ciudad_lineal           107
dto. fict.sec.desap.      4
dtosec.inex. en bdc       1
ciudad lienal             1
Name: count, dtype: int64

Revisando el archivo obtenido, vemos que hay unos valores como 'dto. fict.sec.desap.' 'dtosec.inex. en bdc'. 

Al comprobarlos en los datos originales, comprobamos que no sólo se trata de registros insignificantes de apenas 20 personas, sino que eran erróneos en los archivos obtenidos. Concluimos que fue un error de recogida de datos y decidimos borrarlos.

También hay un error en el que se registró ciudad lineal como 'ciudad lienal', y decidimos enmendarlo también junto con la nomenclatura de san:blas, que debería ser san_blas.

In [94]:
data['distrito'] = data['distrito'].replace({'ciudad lienal':'ciudad_lineal',
                                             'san:blas':'san_blas'})
data = data.drop(data[(data['distrito'] == 'dto. fict.sec.desap.') | (data['distrito'] == 'dtosec.inex. en bdc')].index)

data = data[['distrito', 'nacionales', 'extranjeros', 'total', 'mes']]
data.to_csv('padron_12_22_long.csv', index=False, header=True)



### Limpieza tablas padrón madrid: formato 'wide'

Para pasar esta tabla tipo 'long' (muchas filas, pocas columnas) a un formato 'wide' que podría sernos útil a la hora de realizar determinados cálculos, decidimos trabajar con el .csv que ya acabamos de crear. 

Al importarlo hemos tenido que asegurarnos de nuevo de que nuestra columna 'mes' y nuestras columnas de empadronados estén en el formato adecuado (datetime para 'mes' e int para las demás).

Para pasarla a 'wide' hemos hecho un pivot manteniendo el eje en el mes, y basando las columnas en las diferentes etiquetas de 'distrito', renombrándolas luego (usando la jerarquía resultante de ese pivot) para que el nombre del distrito aparezca primero, seguido de una barra baja '_' y el concepto al que hace referencia la columna.

Posteriormente hemos tenido que organizar las columnas alfabéticamente con un reindex, y ordenar el índice por meses y no por años.

In [95]:

data = pd.read_csv('padron_12_22_long.csv')

data['mes'] = pd.to_datetime(data['mes'], format='%d/%m/%y')
data[['nacionales', 'extranjeros', 'total']] = data[['nacionales', 'extranjeros', 'total']].astype(int)


data_wide = data.pivot(index='mes', columns='distrito', values=['nacionales', 'extranjeros', 'total'])

data_wide.columns = [f'{col[1]}_{col[0]}' for col in data_wide.columns]

data_wide = data_wide.reindex(sorted(data_wide.columns), axis=1)

data_wide = data_wide.sort_index()


data_wide.to_csv('padron_12_22_wide.csv')

### Limpieza tabla alquiler Idealista (long)

Aunque la tabla extraída de idealista ya estaba bastante depurada, aún hay algunos cambios por realizar.

El primero es renombrar la columna 'zona' a 'distrito', para respetar la nomenclatura elegida.

In [35]:
alquiler = pd.read_csv('ALQUILER1223LONG.csv', encoding='latin-1')
alquiler.rename(columns={'zona': 'distrito'}, inplace=True)


In [36]:
alquiler.head()

,mes,precio_m2,var_mensual,var_trimest,var_anual,distrito
0,01/06/23,"16,8 /m2","1,20 %","3,70 %","10,20 %",GLOBAL
1,01/05/23,"16,6 /m2","1,20 %","2,60 %","11,00 %",GLOBAL
2,01/04/23,"16,4 /m2","1,20 %","2,50 %","10,30 %",GLOBAL
3,01/03/23,"16,2 /m2","0,20 %","2,70 %","10,40 %",GLOBAL
4,01/02/23,"16,2 /m2","1,10 %","2,50 %","11,20 %",GLOBAL


Comprobamos los nombres de los distritos y vemos que hay algunos que hay que reemplazar para respetar la nomenclatura acordada. Por ejemplo, 'global' pasará a llamarse 'madrid', o 'villa de vallecas', 'villa_vallecas'.

In [37]:
alquiler['distrito'] = alquiler['distrito'].str.lower()
alquiler.distrito.value_counts()

distrito
global               136
arganzuela           136
villa de vallecas    136
vicalvaro            136
usera                136
tetuan               136
san blas             136
salamanca            136
retiro               136
puente vallecas      136
moratalaz            136
moncloa              136
latina               136
hortaleza            136
fuencarral           136
ciudad lineal        136
chamberi             136
chamartin            136
centro               136
carabanchel          136
barajas              136
villaverde           136
Name: count, dtype: int64

In [99]:
#Reemplazamos los nombres
alquiler['distrito'] = alquiler['distrito'].replace({'global': 'madrid', 
                                                         'villa de vallecas':'villa_vallecas',
                                                         'ciudad lineal':'ciudad_lineal',
                                                         'puente vallecas':'puente_vallecas',
                                                         'san blas':'san_blas'})

Por último, hemos de cambiar las columnas con el formato "1,25 %" para que sean floats que contengan solo el número en cuestión. Para ello primero creamos una lista con los columnas en las que están los valores a cambiar y aplicamos un bucle for para realizar esta transformación.

In [102]:
columnas_cambiar = ['precio_m2','var_mensual','var_trimest', 'var_anual']
for col in columnas_cambiar:
    alquiler[col] = alquiler[col].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)')
    alquiler[col] = alquiler[col].astype(float)

alquiler.to_csv('alquiler_12_23_long.csv', index=False)




### Limpieza tabla alquiler Idealista (wide)

En cuanto a la tabla Wide, los cambios realizados han sido muy parecidos a los de la tabla 'long':

-Cambio de valores numéricos por float.
-Cambio de nombres de columnas a minúscula, cambio de nombre 'global' por 'madrid'


In [30]:
alquiler_w = pd.read_csv('ALQUILER1223WIDE.csv', encoding='latin-1')

In [31]:
alquiler_w.head()

,Mes,GLOBAL_precio_m2,GLOBAL_var_mensual,GLOBAL_var_trimest,GLOBAL_var_anual,ARGANZUELA_precio_m2,ARGANZUELA_var_mensual,ARGANZUELA_var_trimest,ARGANZUELA_var_anual,BARAJAS_precio_m2,...,VICALVARO_var_trimest,VICALVARO_var_anual,VILLA_VALLECAS_precio_m2,VILLA_VALLECAS_var_mensual,VILLA_VALLECAS_var_trimest,VILLA_VALLECAS_var_anual,VILLAVERDE_precio_m2,VILLAVERDE_var_mensual,VILLAVERDE_var_trimest,VILLAVERDE_var_anual
0,01/06/23,"16,8 /m2","1,20 %","3,70 %","10,20 %","16,4 /m2","0,50 %","0,80 %","8,80 %","13,2 /m2",...,"1,80 %","4,00 %","12,5 /m2","-1,30 %","2,20 %","7,00 %","12,3 /m2","0,30 %","4,00 %","9,80 %"
1,01/05/23,"16,6 /m2","1,20 %","2,60 %","11,00 %","16,4 /m2","0,70 %","1,50 %","8,70 %","13,0 /m2",...,"1,10 %","5,80 %","12,6 /m2","-0,90 %","7,00 %","9,40 %","12,3 /m2","2,60 %","3,60 %","10,80 %"
2,01/04/23,"16,4 /m2","1,20 %","2,50 %","10,30 %","16,3 /m2","-0,30 %","0,30 %","7,70 %","12,6 /m2",...,"0,00 %","6,90 %","12,8 /m2","4,50 %","7,40 %","12,60 %","12,0 /m2","1,00 %","2,10 %","9,40 %"
3,01/03/23,"16,2 /m2","0,20 %","2,70 %","10,40 %","16,3 /m2","1,10 %","1,60 %","8,70 %","12,7 /m2",...,"0,10 %","6,70 %","12,2 /m2","3,40 %","2,80 %","9,60 %","11,9 /m2","-0,10 %","0,60 %","10,20 %"
4,01/02/23,"16,2 /m2","1,10 %","2,50 %","11,20 %","16,1 /m2","-0,40 %","0,80 %","7,50 %","12,5 /m2",...,"-0,50 %","7,90 %","11,8 /m2","-0,60 %","0,30 %","5,10 %","11,9 /m2","1,10 %","0,90 %","10,40 %"


In [32]:
lista_nombres = alquiler_w.columns.tolist()
nuevos_nombres = [nombre.lower().replace('global', 'madrid') for nombre in lista_nombres]
alquiler_w.columns = nuevos_nombres


In [33]:
columnas_cambiar = [col for col in nuevos_nombres if col != 'mes']
for col in columnas_cambiar:
    alquiler_w[col] = alquiler_w[col].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)')
    alquiler_w[col] = alquiler_w[col].astype(float)

alquiler_w.to_csv('alquiler_12_23_wide.csv', index=False)
